# ส่วนที่ 4.2: การวิเคราะห์และสร้างตัวอย่างการแสดงผล (Data Visualization) - ตอนที่ 2

Notebook นี้แสดงตัวอย่างการวิเคราะห์ข้อมูลและการสร้างกราฟแบบต่างๆ ที่จะนำไปสร้างใน Power BI (ส่วนที่ 2)

## 1. ติดตั้งแพ็คเกจที่จำเป็น

In [ ]:
# ติดตั้งแพ็คเกจที่จำเป็น
!pip install pandas numpy matplotlib seaborn sqlite3 sqlalchemy plotly

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os
from datetime import datetime
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML

# ตั้งค่าการแสดงผลกราฟให้สวยงาม
plt.style.use('ggplot')
sns.set(style='whitegrid')

# ตั้งค่าให้แสดงข้อมูลภาษาไทยได้
plt.rcParams['font.family'] = 'DejaVu Sans'

# แสดงเวอร์ชันของแพ็คเกจ
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {plt.__version__}")
print(f"Seaborn version: {sns.__version__}")
try:
    print(f"Plotly version: {px.__version__}")
except:
    print("Plotly not installed or version info not available")

## 3. เชื่อมต่อกับฐานข้อมูล SQLite และโหลดข้อมูล

**หมายเหตุ**: คุณต้องรันส่วนที่ 1 (04_Data_Visualization_Part1.ipynb) ก่อนที่จะรันส่วนนี้เพื่อให้แน่ใจว่าฐานข้อมูลและไฟล์ที่จำเป็นถูกสร้างขึ้นแล้ว

In [ ]:
# กำหนดพาธหลักสำหรับโปรเจค
project_path = './Employee_Data_Analytics_Project'  # ปรับตามโครงสร้างของคุณ

# สร้างโฟลเดอร์สำหรับเก็บรูปภาพ (ถ้ายังไม่มี)
viz_folder = f"{project_path}/visualizations"
if not os.path.exists(viz_folder):
    os.makedirs(viz_folder)
    print(f"สร้างโฟลเดอร์ {viz_folder} สำเร็จ")
else:
    print(f"โฟลเดอร์ {viz_folder} มีอยู่แล้ว")

# เชื่อมต่อกับฐานข้อมูล SQLite
db_path = f"{project_path}/employee_database.db"
conn = sqlite3.connect(db_path)
print(f"เชื่อมต่อกับฐานข้อมูล {db_path} สำเร็จ")

# สร้างเอนจินสำหรับ SQLAlchemy
engine = create_engine(f'sqlite:///{db_path}')

# กำหนดชื่อตาราง
employee_table = 'employee_stats'
region_table = 'region_summary'
time_table = 'time_summary'
province_table = 'province_summary'

# โหลดข้อมูลหลัก
df_employee = pd.read_sql_query(f"SELECT * FROM {employee_table}", conn)
print(f"โหลดข้อมูลจากตาราง {employee_table} สำเร็จ: {df_employee.shape[0]} แถว, {df_employee.shape[1]} คอลัมน์")

# ตรวจสอบข้อมูล
print("\nข้อมูลตัวอย่าง:")
display(df_employee.head())

## 4. จัดเตรียมข้อมูลสำหรับการวิเคราะห์เพิ่มเติม

In [ ]:
# สร้างข้อมูลสำหรับการวิเคราะห์ตามที่ต้องการ
# ข้อมูลสรุปตามภูมิภาคและเพศ
region_sex_summary = df_employee.groupby(['REGION', 'SEX'])['VALUE'].sum().reset_index()
region_sex_summary.rename(columns={'VALUE': 'TOTAL_EMPLOYEES'}, inplace=True)

# ข้อมูลแนวโน้มตามเวลา
time_trend = df_employee.groupby(['YEAR', 'QUARTER', 'QUARTER_NUM'])['VALUE'].sum().reset_index()
time_trend = time_trend.sort_values(['YEAR', 'QUARTER_NUM'])
time_trend['YEAR_QUARTER'] = time_trend['YEAR'].astype(str) + '-Q' + time_trend['QUARTER_NUM'].astype(str)

# ข้อมูลตามภูมิภาคและเวลา
region_time = df_employee.groupby(['YEAR', 'QUARTER', 'QUARTER_NUM', 'REGION'])['VALUE'].sum().reset_index()
region_time = region_time.sort_values(['YEAR', 'QUARTER_NUM'])
region_time['YEAR_QUARTER'] = region_time['YEAR'].astype(str) + '-Q' + region_time['QUARTER_NUM'].astype(str)

# แสดงข้อมูลสรุป
print("ข้อมูลสรุปตามภูมิภาคและเพศ:")
display(region_sex_summary.head())

print("\nข้อมูลแนวโน้มตามเวลา:")
display(time_trend.head())

## 5. สร้างกราฟและวิเคราะห์ข้อมูลเพิ่มเติม (ต่อจากส่วนที่ 1)

### 5.1 แผนภูมิแท่งซ้อนทับแสดงสัดส่วนเพศในแต่ละภูมิภาค

In [ ]:
# แผนภูมิแท่งซ้อนทับแสดงสัดส่วนเพศในแต่ละภูมิภาค
region_sex_pct = df_employee.groupby(['REGION', 'SEX'])['VALUE'].sum().reset_index()
region_total = region_sex_pct.groupby('REGION')['VALUE'].sum().reset_index()
region_total.rename(columns={'VALUE': 'TOTAL'}, inplace=True)
region_sex_pct = pd.merge(region_sex_pct, region_total, on='REGION', how='left')
region_sex_pct['PERCENTAGE'] = (region_sex_pct['VALUE'] / region_sex_pct['TOTAL'] * 100).round(1)

# สร้าง pivot table สำหรับการพล็อต
region_sex_pivot = region_sex_pct.pivot(index='REGION', columns='SEX', values='PERCENTAGE').reset_index()
region_sex_pivot = region_sex_pivot.sort_values('ชาย', ascending=False)

plt.figure(figsize=(14, 8))
ax = region_sex_pivot.plot(x='REGION', y=['ชาย', 'หญิง'], kind='bar', stacked=True, figsize=(14, 8),
                          color=['#3498db', '#e74c3c'])

plt.title('สัดส่วนเพศในแต่ละภูมิภาค', fontsize=16)
plt.xlabel('ภูมิภาค', fontsize=12)
plt.ylabel('เปอร์เซ็นต์', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(fontsize=12)

# เพิ่มค่าเปอร์เซ็นต์บนแผนภูมิ
for i, p in enumerate(ax.patches):
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    if height > 5:  # เพิ่มเฉพาะค่าที่มากกว่า 5% เพื่อไม่ให้แน่นเกินไป
        ax.annotate(f'{height:.1f}%', (x + width/2, y + height/2), ha='center', va='center', fontsize=10)

plt.tight_layout()
plt.savefig(f"{viz_folder}/region_sex_stacked_bar_chart.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิแท่งซ้อนทับช่วยให้เห็นทั้งสัดส่วนและการเปรียบเทียบระหว่างกลุ่มย่อยในแต่ละหมวดหมู่หลัก")
print("- สามารถเห็นสัดส่วนร้อยละของพนักงานชายและหญิงในแต่ละภูมิภาคได้อย่างชัดเจน")
print("- แสดงให้เห็นว่าสัดส่วนเพศในแต่ละภูมิภาคมีความใกล้เคียงกันหรือแตกต่างกันอย่างไร")
print("- แนะนำให้ใช้แผนภูมิแท่งซ้อนทับใน Power BI สำหรับการแสดงสัดส่วนของกลุ่มย่อยในแต่ละหมวดหมู่หลัก")

### 5.2 แผนภูมิความร้อนแสดงจำนวนพนักงานตามจังหวัดและเพศ

In [ ]:
# แผนภูมิความร้อนแสดงจำนวนพนักงานตามจังหวัดและเพศ
# เลือกเฉพาะ 15 จังหวัดที่มีจำนวนพนักงานมากที่สุด
province_total = df_employee.groupby('PROVINCE')['VALUE'].sum().reset_index()
top_15_provinces = province_total.sort_values('VALUE', ascending=False).head(15)['PROVINCE'].tolist()

# สร้างข้อมูลสำหรับพล็อต
province_sex = df_employee[df_employee['PROVINCE'].isin(top_15_provinces)].groupby(['PROVINCE', 'SEX'])['VALUE'].mean().reset_index()
province_sex_pivot = province_sex.pivot(index='PROVINCE', columns='SEX', values='VALUE').reset_index()
province_sex_pivot = province_sex_pivot.sort_values('ชาย', ascending=False)

# สร้างแผนภูมิความร้อน
plt.figure(figsize=(12, 10))
data = province_sex_pivot.set_index('PROVINCE')
ax = sns.heatmap(data, annot=True, fmt='.1f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'จำนวนพนักงานเฉลี่ย (พันคน)'})
plt.title('จำนวนพนักงานเฉลี่ยตามจังหวัดและเพศ (15 จังหวัดแรก)', fontsize=16)
plt.xlabel('เพศ', fontsize=12)
plt.ylabel('จังหวัด', fontsize=12)

plt.tight_layout()
plt.savefig(f"{viz_folder}/province_sex_heatmap.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิความร้อนช่วยให้เห็นความเข้มของข้อมูลระหว่างมิติต่างๆ ผ่านการใช้สี")
print("- สามารถเปรียบเทียบจำนวนพนักงานเฉลี่ยระหว่างจังหวัดและเพศได้อย่างชัดเจน")
print("- เห็นได้ว่าจังหวัดที่มีจำนวนพนักงานเฉลี่ยสูงสุดคือ กรุงเทพมหานคร ทั้งในกลุ่มพนักงานชายและหญิง")
print("- แนะนำให้ใช้แผนภูมิความร้อนใน Power BI สำหรับการแสดงความเข้มของข้อมูลระหว่างมิติต่างๆ")

### 5.3 แผนภูมิเส้นและแท่งผสม (Combo Chart) แสดงจำนวนพนักงานและการเปลี่ยนแปลง

In [ ]:
# แผนภูมิเส้นและแท่งผสม (Combo Chart) แสดงจำนวนพนักงานและการเปลี่ยนแปลง
# สร้างข้อมูลสำหรับพล็อต
combo_data = time_trend.copy()
combo_data['PREV_VALUE'] = combo_data['VALUE'].shift(1)
combo_data['CHANGE_PCT'] = ((combo_data['VALUE'] - combo_data['PREV_VALUE']) / combo_data['PREV_VALUE'] * 100).round(1)
combo_data = combo_data.dropna()

# สร้างแผนภูมิผสม
fig, ax1 = plt.subplots(figsize=(14, 6))

# แกน Y ซ้าย (แผนภูมิแท่ง - จำนวนพนักงาน)
bar_plot = ax1.bar(combo_data['YEAR_QUARTER'], combo_data['VALUE'], color='#3498db', alpha=0.7)
ax1.set_xlabel('ปี-ไตรมาส', fontsize=12)
ax1.set_ylabel('จำนวนพนักงาน (พันคน)', fontsize=12, color='#3498db')
ax1.tick_params(axis='y', labelcolor='#3498db')
plt.xticks(rotation=45)

# แกน Y ขวา (แผนภูมิเส้น - เปอร์เซ็นต์การเปลี่ยนแปลง)
ax2 = ax1.twinx()
line_plot = ax2.plot(combo_data['YEAR_QUARTER'], combo_data['CHANGE_PCT'], 'r-', marker='o', linewidth=2, color='#e74c3c')
ax2.set_ylabel('การเปลี่ยนแปลง (%)', fontsize=12, color='#e74c3c')
ax2.tick_params(axis='y', labelcolor='#e74c3c')

# เส้นแสดงการเปลี่ยนแปลง 0%
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# เพิ่มสีให้กับการเปลี่ยนแปลงเป็นบวกหรือลบ
for i, val in enumerate(combo_data['CHANGE_PCT']):
    color = 'green' if val >= 0 else 'red'
    ax2.annotate(f'{val}%', 
                xy=(i, val), 
                xytext=(0, 5 if val >= 0 else -15),  # แยกแต่ละค่าให้ไม่ทับกัน
                textcoords='offset points',
                ha='center', 
                va='bottom' if val >= 0 else 'top',
                color=color,
                fontsize=9)

plt.title('จำนวนพนักงานและเปอร์เซ็นต์การเปลี่ยนแปลงตามไตรมาส', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.3)

# เพิ่ม legend
ax1.legend(['จำนวนพนักงาน'], loc='upper left')
ax2.legend(['การเปลี่ยนแปลง (%)'], loc='upper right')

plt.tight_layout()
plt.savefig(f"{viz_folder}/employee_change_combo_chart.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิผสมช่วยให้เห็นความสัมพันธ์ระหว่างตัวแปรที่มีหน่วยแตกต่างกัน")
print("- สามารถเห็นทั้งจำนวนพนักงานและเปอร์เซ็นต์การเปลี่ยนแปลงในแผนภูมิเดียวกัน")
print("- เห็นได้ว่าเปอร์เซ็นต์การเปลี่ยนแปลงมีความผันผวนมากในบางไตรมาส")
print("- แนะนำให้ใช้แผนภูมิผสมใน Power BI สำหรับการแสดงตัวแปรที่มีหน่วยวัดแตกต่างกันในแผนภูมิเดียวกัน")

### 5.4 แผนภูมิพื้นที่แสดงสัดส่วนพนักงานตามภูมิภาคและเวลา

In [ ]:
# แผนภูมิพื้นที่แสดงสัดส่วนพนักงานตามภูมิภาคและเวลา
# สร้าง pivot table สำหรับการพล็อต
region_time_pivot = region_time.pivot(index='YEAR_QUARTER', columns='REGION', values='VALUE')

# สร้างแผนภูมิพื้นที่
plt.figure(figsize=(14, 8))
ax = region_time_pivot.plot.area(figsize=(14, 8), alpha=0.7, cmap='viridis')

plt.title('สัดส่วนพนักงานตามภูมิภาคและไตรมาส', fontsize=16)
plt.xlabel('ปี-ไตรมาส', fontsize=12)
plt.ylabel('จำนวนพนักงาน (พันคน)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.3)
plt.legend(title='ภูมิภาค', loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.savefig(f"{viz_folder}/region_time_area_chart.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิพื้นที่เป็นเครื่องมือที่เหมาะสมสำหรับการแสดงสัดส่วนและแนวโน้มตามช่วงเวลา")
print("- สามารถเห็นการเปลี่ยนแปลงของสัดส่วนพนักงานในแต่ละภูมิภาคตามช่วงเวลา")
print("- เห็นได้ว่าสัดส่วนระหว่างภูมิภาคค่อนข้างคงที่ แม้ว่าจำนวนรวมจะมีการเปลี่ยนแปลง")
print("- แนะนำให้ใช้แผนภูมิพื้นที่ใน Power BI สำหรับการแสดงสัดส่วนที่เปลี่ยนแปลงตามเวลา")

### 5.5 แผนภูมิจุดแสดงความสัมพันธ์ระหว่างจำนวนพนักงานชายและหญิง

In [ ]:
# แผนภูมิจุดแสดงความสัมพันธ์ระหว่างจำนวนพนักงานชายและหญิง
scatter_data = df_employee.groupby(['PROVINCE', 'YEAR', 'QUARTER'])['VALUE'].sum().reset_index()
scatter_data = scatter_data.pivot_table(index=['PROVINCE', 'YEAR', 'QUARTER'], columns='SEX', values='VALUE').reset_index()
scatter_data = scatter_data.dropna()

# สร้างแผนภูมิจุด
plt.figure(figsize=(12, 10))
plt.scatter(scatter_data['ชาย'], scatter_data['หญิง'], alpha=0.5, s=50, c=scatter_data['YEAR'], cmap='viridis')

plt.title('ความสัมพันธ์ระหว่างจำนวนพนักงานชายและหญิงตามจังหวัด', fontsize=16)
plt.xlabel('จำนวนพนักงานชาย (พันคน)', fontsize=12)
plt.ylabel('จำนวนพนักงานหญิง (พันคน)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.colorbar(label='ปี')

# เพิ่มเส้นแสดงความสัมพันธ์ 1:1 (จำนวนชายเท่ากับหญิง)
max_val = max(scatter_data['ชาย'].max(), scatter_data['หญิง'].max())
plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.7, label='สัดส่วน 1:1')

# เพิ่ม regression line
z = np.polyfit(scatter_data['ชาย'], scatter_data['หญิง'], 1)
p = np.poly1d(z)
plt.plot(np.sort(scatter_data['ชาย']), p(np.sort(scatter_data['ชาย'])), "g-", alpha=0.7, label=f'Trend Line (y={z[0]:.2f}x+{z[1]:.2f})')

plt.legend(fontsize=10)
plt.tight_layout()
plt.savefig(f"{viz_folder}/gender_relation_scatter_plot.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิจุดช่วยให้เห็นความสัมพันธ์ระหว่างตัวแปรสองตัว")
print("- สามารถเห็นความสัมพันธ์ระหว่างจำนวนพนักงานชายและหญิงในแต่ละจังหวัดและช่วงเวลา")
print("- เส้นประสีแดงแสดงสัดส่วน 1:1 (จำนวนชายเท่ากับหญิง) ส่วนเส้นสีเขียวแสดงแนวโน้มความสัมพันธ์จริง")
print("- ส่วนใหญ่จุดอยู่เหนือเส้นประสีแดง แสดงว่าจำนวนพนักงานชายมักจะมากกว่าพนักงานหญิง")
print("- แนะนำให้ใช้แผนภูมิจุดใน Power BI สำหรับการแสดงความสัมพันธ์ระหว่างตัวแปรสองตัว")

### 5.6 แผนภูมิกล่อง (Box Plot) แสดงการกระจายตัวของจำนวนพนักงานตามภูมิภาค

In [ ]:
# แผนภูมิกล่อง (Box Plot) แสดงการกระจายตัวของจำนวนพนักงานตามภูมิภาค
plt.figure(figsize=(14, 8))
ax = sns.boxplot(x='REGION', y='VALUE', data=df_employee, palette='viridis')
plt.title('การกระจายตัวของจำนวนพนักงานตามภูมิภาค', fontsize=16)
plt.xlabel('ภูมิภาค', fontsize=12)
plt.ylabel('จำนวนพนักงาน (พันคน)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig(f"{viz_folder}/region_box_plot.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิกล่องแสดงการกระจายตัวของข้อมูลและช่วยในการระบุค่าผิดปกติ (Outliers)")
print("- สามารถเห็นค่ามัธยฐาน (เส้นตรงกลางกล่อง) และช่วงระหว่างควอร์ไทล์ที่ 1 ถึงควอร์ไทล์ที่ 3 (ความสูงของกล่อง)")
print("- จุดที่อยู่นอกขอบบน-ล่างแสดงถึงค่าผิดปกติ")
print("- แนะนำให้ใช้แผนภูมิกล่องใน Power BI สำหรับการวิเคราะห์การกระจายตัวของข้อมูล")

## 6. สรุปผลการวิเคราะห์ข้อมูลและการสร้างแผนภูมิ

In [ ]:
# สรุปผลการวิเคราะห์ข้อมูลและการสร้างแผนภูมิ
print("สรุปผลการวิเคราะห์ข้อมูลและการสร้างแผนภูมิ:")
print("\n1. การเปรียบเทียบข้อมูลระหว่างกลุ่ม:")
print("- แผนภูมิแท่ง: ใช้เปรียบเทียบค่าระหว่างภูมิภาค")
print("- แผนภูมิแท่งจัดกลุ่ม: ใช้เปรียบเทียบจำนวนพนักงานชายและหญิงในแต่ละภูมิภาค")
print("- แผนภูมิแท่งซ้อนทับ: ใช้แสดงสัดส่วนเพศในแต่ละภูมิภาค")
print("- แผนภูมิกล่อง: ใช้แสดงการกระจายตัวของข้อมูลและค่าผิดปกติ")

print("\n2. การแสดงสัดส่วน:")
print("- แผนภูมิวงกลม: ใช้แสดงสัดส่วนพนักงานตามเพศ")
print("- แผนภูมิพื้นที่: ใช้แสดงสัดส่วนพนักงานตามภูมิภาคและเวลา")

print("\n3. การแสดงแนวโน้มตามเวลา:")
print("- แผนภูมิเส้น: ใช้แสดงแนวโน้มการจ้างงานตามเวลา")
print("- แผนภูมิเส้นหลายเส้น: ใช้เปรียบเทียบแนวโน้มการจ้างงานระหว่างเพศชายและหญิง")
print("- แผนภูมิผสม: ใช้แสดงจำนวนพนักงานและเปอร์เซ็นต์การเปลี่ยนแปลงในแผนภูมิเดียวกัน")

print("\n4. การแสดงความสัมพันธ์:")
print("- แผนภูมิจุด: ใช้แสดงความสัมพันธ์ระหว่างจำนวนพนักงานชายและหญิง")
print("- แผนภูมิความร้อน: ใช้แสดงความเข้มของข้อมูลระหว่างมิติต่างๆ")

print("\n5. ข้อควรพิจารณาเมื่อสร้าง Dashboard ใน Power BI:")
print("- เลือกใช้ Visualization ที่เหมาะสมกับประเภทของข้อมูลและวัตถุประสงค์ในการนำเสนอ")
print("- ใช้สีที่สื่อความหมายและสอดคล้องกัน เช่น สีเขียวแสดงค่าบวก สีแดงแสดงค่าลบ")
print("- จัดวางองค์ประกอบให้เป็นระเบียบและอ่านง่าย โดยเรียงลำดับตามความสำคัญ")
print("- เพิ่มตัวกรอง (Filters) หรือ Slicers เพื่อให้ผู้ใช้สามารถโต้ตอบกับข้อมูลได้")
print("- แสดงค่าสำคัญบน Visual เพื่อให้ผู้ใช้เห็นได้ทันที")

## 7. บันทึกข้อมูลวิเคราะห์สำหรับใช้ใน Power BI

In [ ]:
# บันทึกข้อมูลวิเคราะห์สำหรับใช้ใน Power BI
visualization_summary = {
    'chart_types': [
        {
            'type': 'Bar Chart',
            'usage': 'เปรียบเทียบค่าระหว่างกลุ่ม',
            'example': 'จำนวนพนักงานตามภูมิภาค',
            'file': 'region_bar_chart.png'
        },
        {
            'type': 'Pie Chart',
            'usage': 'แสดงสัดส่วน',
            'example': 'สัดส่วนพนักงานตามเพศ',
            'file': 'gender_pie_chart.png'
        },
        {
            'type': 'Line Chart',
            'usage': 'แสดงแนวโน้มตามเวลา',
            'example': 'แนวโน้มการจ้างงานตามไตรมาส',
            'file': 'time_trend_line_chart.png'
        },
        {
            'type': 'Combo Chart',
            'usage': 'แสดงหลายตัวแปรที่มีหน่วยต่างกัน',
            'example': 'จำนวนพนักงานและเปอร์เซ็นต์การเปลี่ยนแปลง',
            'file': 'employee_change_combo_chart.png'
        },
        {
            'type': 'Heatmap',
            'usage': 'แสดงความเข้มของข้อมูลระหว่างมิติต่างๆ',
            'example': 'จำนวนพนักงานตามจังหวัดและเพศ',
            'file': 'province_sex_heatmap.png'
        },
        {
            'type': 'Stacked Bar Chart',
            'usage': 'แสดงสัดส่วนในแต่ละกลุ่ม',
            'example': 'สัดส่วนเพศในแต่ละภูมิภาค',
            'file': 'region_sex_stacked_bar_chart.png'
        },
        {
            'type': 'Area Chart',
            'usage': 'แสดงสัดส่วนตามเวลา',
            'example': 'สัดส่วนพนักงานตามภูมิภาคและไตรมาส',
            'file': 'region_time_area_chart.png'
        },
        {
            'type': 'Scatter Plot',
            'usage': 'แสดงความสัมพันธ์ระหว่างตัวแปร',
            'example': 'ความสัมพันธ์ระหว่างจำนวนพนักงานชายและหญิง',
            'file': 'gender_relation_scatter_plot.png'
        },
        {
            'type': 'Box Plot',
            'usage': 'แสดงการกระจายตัวของข้อมูล',
            'example': 'การกระจายตัวของจำนวนพนักงานตามภูมิภาค',
            'file': 'region_box_plot.png'
        }
    ]
}

# บันทึกเป็นไฟล์ JSON
import json
with open(f"{viz_folder}/visualization_summary.json", 'w', encoding='utf-8') as f:
    json.dump(visualization_summary, f, ensure_ascii=False, indent=4)

print(f"บันทึกข้อมูลสรุปการวิเคราะห์ไปยัง {viz_folder}/visualization_summary.json สำเร็จ")

# ปิดการเชื่อมต่อกับฐานข้อมูล
conn.close()
print("ปิดการเชื่อมต่อกับฐานข้อมูลสำเร็จ")